In [ ]:
#| default_exp prune.pruner

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_pruning as tp
from torch_pruning.pruner import function

import numpy as np
import torch
import torch.nn as nn
import pickle
from itertools import cycle
from fastcore.basics import store_attr, listify, true
from fasterai.core.criteria import *
from fastai.vision.all import *


from torch_pruning.pruner.algorithms.scheduler import linear_scheduler
from torch.fx import symbolic_trace

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
class Pruner():
    def __init__(self, model, pruning_ratio, context, criteria, schedule=linear_scheduler, ignored_layers=None, example_inputs=torch.randn(1, 3, 224, 224), *args, **kwargs):
        store_attr()
        self.num_heads = {}
        if not self.ignored_layers: self.get_ignored_layers(self.model)
        if self.pruning_ratio>1: self.pruning_ratio = self.pruning_ratio/100 
        self.pruner = tp.pruner.MetaPruner(
        self.model,
        example_inputs=self.example_inputs.to(next(self.model.parameters()).device),
        importance=self.group_importance,
        pruning_ratio=self.pruning_ratio, 
        ignored_layers=self.ignored_layers,
        global_pruning=True if self.context=='global' else False,
        num_heads = self.num_heads,
        iterative_pruning_ratio_scheduler=self.schedule,
        *args, 
        **kwargs
        )
          
    def prune_model(self):
        self.pruner.step()
        self.restore_attention_layers()


    def get_linear_layers_to_ignore(self, model):
        traced = symbolic_trace(model)
        for node in traced.graph.nodes:
            if node.op == "output":  # Identifier la sortie
                for input_node in node.all_input_nodes:
                    if input_node.target:  # Trouver la couche correspondante
                        module = dict(model.named_modules()).get(input_node.target)
                        if isinstance(module, torch.nn.Linear):
                            self.ignored_layers.append(module)
                            print(f"Ignoring output layer: {module}")


    def get_attention_layers_to_ignore(self, model):
        for module in model.modules():
            if hasattr(module, 'num_heads'):
                if hasattr(module, 'qkv'):
                    self.ignored_layers.append(module.qkv)  # Ajouter à la liste globale
                    self.num_heads[module.qkv] = module.num_heads
                    print(f"Attention layer ignored: {module.qkv}, num_heads={module.num_heads}")
                elif hasattr(module, 'qkv_proj'):
                    self.ignored_layers.append(module.qkv_proj)  # Ajouter à la liste globale
                    self.num_heads[module.qkv_proj] = module.num_heads
                    print(f"Attention layer ignored: {module.qkv_proj}, num_heads={module.num_heads}")

    
    def get_ignored_layers(self, model):
        self.ignored_layers = []
        self.get_linear_layers_to_ignore(model)
        self.get_attention_layers_to_ignore(model)
        print(f"Total ignored layers: {len(self.ignored_layers)}")
    
                
    def restore_attention_layers(self):
        for m in self.model.modules():
            # Attention layers
            if hasattr(m, 'num_heads'):
                if hasattr(m, 'qkv'):
                    m.num_heads = self.num_heads[m.qkv]
                    m.head_dim = m.qkv.out_features // (3 * m.num_heads)
                elif hasattr(m, 'qkv_proj'):
                    m.num_heads = self.num_heads[m.qqkv_projkv]
                    m.head_dim = m.qkv_proj.out_features // (3 * m.num_heads)


    def group_importance(self, group):
        handler_map = {
            function.prune_conv_out_channels: 'filter',
            function.prune_linear_out_channels: 'row',
            function.prune_linear_in_channels: 'column',
            function.prune_conv_in_channels: 'shared_kernel',
            # Additional handlers can be added here
        }
    
        group_imp = []
        group_idxs = []
    
        for i, (dep, idxs) in enumerate(group):
            if dep.handler in handler_map:
                impo = self.criteria(dep.target.module, handler_map.get(dep.handler), squeeze=True)
                group_imp.append(impo)
                group_idxs.append(group[i].root_idxs)
    
        reduced_imp = torch.zeros_like(group_imp[0])
    
        for i, (imp, root_idxs) in enumerate(zip(group_imp, group_idxs)):
            imp = imp.to('cpu')
            reduced_imp = reduced_imp.to('cpu')
            reduced_imp.scatter_add_(0, torch.tensor(root_idxs, device=imp.device), imp)
    
        reduced_imp /= len(group_imp)
    
        return reduced_imp.to(default_device())

In [ ]:
show_doc(Pruner.prune_model)

---

[source](https://github.com/FasterAI-Labs/fasterai/tree/master/blob/master/fasterai/prune/pruner.py#L46){target="_blank" style="float:right; font-size:smaller"}

### Pruner.prune_model

>      Pruner.prune_model ()

Let's try the `Pruner` with a VGG16 model

In [ ]:
model = resnet18(); model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

The `Pruner`can either remove filters based on `local` criteria (i.e. each layer will be trimmed of the same % of filters)

In [ ]:
pruner = Pruner(model, 30, 'local', large_final)
pruner.prune_model()
print(model)

Ignoring output layer: Linear(in_features=512, out_features=1000, bias=True)
Total ignored layers: 1
ResNet(
  (conv1): Conv2d(3, 44, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(44

The `Pruner`can also remove filters based on `global` criteria (i.e. each layer will be trimmed of a different % of filters, but we specify the sparsity of the whole network)

In [ ]:
model = resnet18()
pruner = Pruner(model, 50, 'global', large_final)
pruner.prune_model()
print(model)

Ignoring output layer: Linear(in_features=512, out_features=1000, bias=True)
Total ignored layers: 1
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64

In [ ]:
from fasterai.core.all import *

In [ ]:
model = resnet18()

In [ ]:
pruner = Pruner(model, 30, 'local', large_final, schedule=one_cycle._scheduler, iterative_steps=3)

Ignoring output layer: Linear(in_features=512, out_features=1000, bias=True)
Total ignored layers: 1


In [ ]:
pruner.pruning_ratio

0.3

In [ ]:
pruner.prune_model()
print(model)

ResNet(
  (conv1): Conv2d(3, 44, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(44, 44, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(44, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

/home/HubensN/miniconda3/envs/fasterai20/lib/python3.10/site-packages/torch_pruning/pruner/algorithms/metapruner.py:393: UserWarning: Pruning exceed the maximum iterative steps, no pruning will be performed.
  warnings.warn("Pruning exceed the maximum iterative steps, no pruning will be performed.")


In [ ]:
%debug

> /home/HubensN/miniconda3/envs/fasterai20/lib/python3.10/site-packages/torch_pruning/pruner/algorithms/metapruner.py(467)_prune()
    465                     continue
    466 
--> 467                 record = (group, ch_groups, group_size, self.per_step_pruning_ratio[self.current_step], dim_imp) # otherwise, use the default pruning ratio
    468                 # Scope 2: Isomorphic Pruning
    469                 if self.isomorphic:



ipdb>  self.current_step


1


ipdb>  self.per_step_pruning_ratio


[0.0]


ipdb>  quit
